In [1]:
import re
import requests
import urllib.request
from urllib.parse import urljoin

from bs4 import BeautifulSoup

import pandas as pd

from tqdm import tqdm
tqdm.pandas()

In [6]:
df = pd.read_hdf('../metadata.h5')
df.head(50)

,date_added,date_modified,guid,id,meta_info,name,parent,type,url,keyword,img_url
0,13202479594197706,13242831177794171,00000000-0000-4000-a000-000000000002,1,,書籤列,bookmark_bar,folder,,[],None
1,13233003241034750,13233003241037911,f0b52077-6793-498a-b8eb-462dfc400922,1884,,2019-nCoV,書籤列,folder,,"[2019, nCoV]",None
2,13226238181043741,,7f8a21b9-b559-44ab-b711-c41800d188d1,1885,,Coronavirus_Taiwan,2019-nCoV,url,https://viator.maps.arcgis.com/apps/opsdashboa...,"[Coronavirus, Taiwan]",None
3,13226407607803828,,24a7d7be-2a87-4481-b157-3773db2f4eab,1886,,2019-nCoV | 武漢肺炎地圖,2019-nCoV,url,https://kiang.github.io/2019-nCoV/,"[2019, nCoV, 武漢, 地圖, 肺炎]",None
4,13227360115873492,,a09ae3f0-3a5f-417c-ad10-423d35e155f3,1888,,程序員硬核勸告：現在還不是出門的時候 - 幫趣,2019-nCoV,url,https://bangqu.com/a7wXH5.html,"[硬核, 勸告, 現在還, 出門, 時候, 幫趣, 程序]",None
5,13232695304731928,,8c555a82-3c7b-4f14-8f19-9cfe663e5d19,1890,,300新竹市東區高翠路173巷4弄 - Google 地圖,2019-nCoV,url,https://www.google.com/maps/place/300%E6%96%B0...,"[300, 東區, 高翠路, 173, Google, 地圖, 新竹市]",None
6,13233003241047126,13242239034086184,90b08b84-66d4-488d-8e18-20cdf3ba014a,1891,,木刻思,書籤列,folder,,[木刻],None
7,13228386784256522,,8d719a89-0635-42d2-a750-52ab77bd8d8f,1892,,Hubstaff - My account: Billing,木刻思,url,https://app.hubstaff.com/users/796279/billing,"[Hubstaff, My, account, Billing]",None
8,13228386905254476,,ab84fcbe-ce9f-4823-b200-9066e72d3896,1893,,[HQ] New Member On Board,木刻思,url,https://3.basecamp.com/3287947/projects/4147783,"[HQ, New, Member, Board]",None
9,13230802944793614,,b66208e6-5322-4eac-9841-bb50844182ee,1894,,iiNumbers HQ / Business Development Office / D...,木刻思,url,https://gitlab.iinumbers.net/iinumbers-hq/bdo/...,"[Project, iiNumbers, HQ, Business, Development...",None


In [7]:
def get_thumbnail(row):
    """去找網站裡面已經預設好的 thumbnail，結果會有下面三種情形：
    
        1. None: 沒有結果，之後會重跑
        2. '': 有連到，但判定為沒有 thumbnail
        3. 'http:...': 成功找到 thumbnail 網址
    """

    if not row['url']:
        return ''

    if row['img_url'] == None:
        try:
            with requests.get(row['url'], timeout=1.5) as res:
                html = BeautifulSoup(res.text, 'html.parser')
            img_node = html.find('meta', attrs={"property":"og:image"})
            if img_node:
                return img_node.get('content')
            else:
                return ''
        except Exception as e:
            return None

for i, row in tqdm(list(df.iterrows())):
    df.loc[i, 'img_url'] = get_thumbnail(row)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 521/521 [11:46<00:00,  1.01s/it]

In [8]:
df.head()

,date_added,date_modified,guid,id,meta_info,name,parent,type,url,keyword,img_url
0,13202479594197706,13242831177794171,00000000-0000-4000-a000-000000000002,1,,書籤列,bookmark_bar,folder,,[],
1,13233003241034750,13233003241037911,f0b52077-6793-498a-b8eb-462dfc400922,1884,,2019-nCoV,書籤列,folder,,"[2019, nCoV]",
2,13226238181043741,,7f8a21b9-b559-44ab-b711-c41800d188d1,1885,,Coronavirus_Taiwan,2019-nCoV,url,https://viator.maps.arcgis.com/apps/opsdashboa...,"[Coronavirus, Taiwan]",
3,13226407607803828,,24a7d7be-2a87-4481-b157-3773db2f4eab,1886,,2019-nCoV | 武漢肺炎地圖,2019-nCoV,url,https://kiang.github.io/2019-nCoV/,"[2019, nCoV, 武漢, 地圖, 肺炎]",http://kiang.github.io/2019-nCoV/og_image.png
4,13227360115873492,,a09ae3f0-3a5f-417c-ad10-423d35e155f3,1888,,程序員硬核勸告：現在還不是出門的時候 - 幫趣,2019-nCoV,url,https://bangqu.com/a7wXH5.html,"[硬核, 勸告, 現在還, 出門, 時候, 幫趣, 程序]",http://i2.bangqu.com/liang/20200205/v2-3a69d59...


In [9]:
df.to_hdf('../metadata.h5', key='data')

C:\Users\adam8\Anaconda3\lib\site-packages\pandas\core\generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['date_added', 'date_modified', 'guid', 'id', 'meta_info', 'name', 'parent', 'type', 'url', 'keyword', 'img_url']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [10]:
def get_image(row):
    if row['img_url']:
        img_url = urljoin(*row[['url', 'img_url']])

        try:
            urllib.request.urlretrieve(img_url, f"../static/images/{row['guid']}")
                
        except Exception as e:
            print(e)

df.progress_apply(get_image, axis='columns')


  5%|██████████▋                                                                                                                                                                                           | 28/521 [00:08<04:29,  1.83it/s]

HTTP Error 403: Forbidden



  6%|████████████▌                                                                                                                                                                                         | 33/521 [00:10<02:43,  2.99it/s]

HTTP Error 403: Forbidden



 12%|████████████████████████▎                                                                                                                                                                             | 64/521 [00:15<01:34,  4.85it/s]

...


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍ | 517/521 [02:23<00:02,  1.81it/s]

HTTP Error 403: Forbidden



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 521/521 [02:25<00:00,  1.93it/s]

HTTP Error 403: Forbidden


0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
21     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
       ... 
491    None
492    None
493    None
494    None
495    None
496    None
497    None
498    None
499    None
500    None
501    None
502    None
503    None
504    None
505    None
506    None
507    None
508    None
509    None
510    None
511    None
512    None
513    None
514    None
515    None
516    None
517    None
518    None
519    None
520    None
Length: 521, dtype: object